# OCaml Project INF231: How to vote ?

## 2. Scrutin uninominal 

### Question 1

In [12]:
type candidat = string ;;
type bulletin = string ;;
type urne = string list ;;
type score = int (* Between 0 and 16*) ;;
type panel = string list ;;

type candidat = string


type bulletin = string


type urne = string list


type score = int


type panel = string list


### Question 2

In [13]:
let rec compte (c: candidat) (u: urne ) : score =
    match u with
        | [] -> 0
        | a::remainder -> 
            if a = c then 1 + compte c remainder 
            else compte c remainder

(*TEST DE FONCTION AVEC JEU D'ESSAIS*)

let lc1 = ["Eric";"Kyle";"Stan"]
let u1 = ["Eric";"Kyle";"Stan";"Kyle";"Kyle";"Stan";"Eric";"Eric";"Kyle";"Eric";"Stan";"Eric";
"Eric";"Eric";"Stan";"Stan"];;

compte "Eric" u1 ;;

val compte : candidat -> urne -> score = <fun>


val lc1 : string list = ["Eric"; "Kyle"; "Stan"]


val u1 : string list =
  ["Eric"; "Kyle"; "Stan"; "Kyle"; "Kyle"; "Stan"; "Eric"; "Eric"; "Kyle";
   "Eric"; "Stan"; "Eric"; "Eric"; "Eric"; "Stan"; "Stan"]


- : score = 7


### Question 3

In [14]:
type resultat = (candidat * score) list ;;

let rec depouiller (lc: panel) (u : urne) : resultat =
    match lc with
        | [] -> []
        | a::remainder ->
            let c = compte a u in
                [(a,c)] @ (depouiller remainder u);;
                
(*TEST DE FONCTION AVEC JEU D'ESSAIS*)

depouiller lc1 u1 ;;

    

type resultat = (candidat * score) list


val depouiller : panel -> urne -> resultat = <fun>


- : resultat = [("Eric", 7); ("Kyle", 4); ("Stan", 5)]


### Question 4

In [15]:
let rec union (r1: resultat) (r2: resultat) : resultat =
    match r1 with
    | [] -> r2
    | (c1,s1)::t1 ->
        match r2 with
        | [] -> r1
        | (c2,s2)::t2 ->
            if c1 = c2 then
                (c1, s1 + s2) :: union t1 t2
            else if c1 < c2 then
                (c1, s1) :: union t1 r2
            else
                (c2, s2) :: union r1 t2;;

(*TEST DE FONCTION AVEC JEU D'ESSAIS*)

let u2 = ["Stan"; "Eric"; "Eric";"Kyle" ; "Stan" ; "Stan" ; "Eric" ; "Kyle"];;

let r1 = depouiller lc1 u1 ;;
let r2 = depouiller lc1 u2 ;; 

union r1 r2 ;;

val union : resultat -> resultat -> resultat = <fun>


val u2 : string list =
  ["Stan"; "Eric"; "Eric"; "Kyle"; "Stan"; "Stan"; "Eric"; "Kyle"]


val r1 : resultat = [("Eric", 7); ("Kyle", 4); ("Stan", 5)]


val r2 : resultat = [("Eric", 3); ("Kyle", 2); ("Stan", 3)]


- : resultat = [("Eric", 10); ("Kyle", 6); ("Stan", 8)]


### Question 5

In [16]:
let rec max_depouille (l : resultat) : candidat * score =
  match l with
  | [] -> failwith "Empty list"
  | [(c,s)] -> (c,s)
  | (c1,s1)::(c2,s2)::remainder ->
      if s1 >= s2 then max_depouille ((c1,s1)::remainder)
      else max_depouille ((c2,s2)::remainder) ;;

(*TEST DE FONCTION AVEC JEU D'ESSAIS*)

max_depouille r1 ;;

         

val max_depouille : resultat -> candidat * score = <fun>


- : candidat * score = ("Eric", 7)


### Question 6

In [17]:
let vainqueur_scrutin_uninominal (u : urne) (lc : panel) : candidat = 
    let r1 = depouiller lc u in 
        let (c, s) = max_depouille r1 in
            c ;;

(*TEST DE FONCTION AVEC JEU D'ESSAIS*)
            
vainqueur_scrutin_uninominal u1 lc1 ;;

val vainqueur_scrutin_uninominal : urne -> panel -> candidat = <fun>


- : candidat = "Eric"


### Question 7

In [18]:
let rec suppr_elem (l : 'a list) (e : 'a) : 'a list =
    match l with 
        | [] -> []
        | a::remainder -> if a = e then remainder else a::(suppr_elem remainder e) ;;

                    
let deux_premiers (u: urne) (lc: panel) : (candidat * score) * (candidat * score) =
  let r = depouiller lc u in
      let (c1, s1) = max_depouille r in
          let r' = depouiller (suppr_elem lc c1) u in
              let (c2, s2) = max_depouille r' in
                  ((c1, s1), (c2, s2)) ;;

(*TEST*)

deux_premiers u1 lc1 ;;
    

val suppr_elem : 'a list -> 'a -> 'a list = <fun>


val deux_premiers : urne -> panel -> (candidat * score) * (candidat * score) =
  <fun>


- : (candidat * score) * (candidat * score) = (("Eric", 7), ("Stan", 5))


### Question 8

In [39]:
let lc3 = ["Eric";"Kyle";"Stan";"Keny"];;
let u3 = ["Keny";"Kyle";"Keny";"Kyle";"Kyle";"Keny";"Eric";"Eric";"Kyle";"Eric";"Stan";"Eric";
"Eric";"Eric";"Stan";"Stan"];;

(*Afin de mieux analyser le paradoxe d'Arrox, un test avec un jeu d'essais:*)

deux_premiers u3 lc3 ;;
depouiller lc3 u3 ;;

val lc3 : string list = ["Eric"; "Kyle"; "Stan"; "Keny"]


val u3 : string list =
  ["Keny"; "Kyle"; "Keny"; "Kyle"; "Kyle"; "Keny"; "Eric"; "Eric"; "Kyle";
   "Eric"; "Stan"; "Eric"; "Eric"; "Eric"; "Stan"; "Stan"]


- : (candidat * score) * (candidat * score) = (("Eric", 6), ("Kyle", 4))


- : resultat = [("Eric", 6); ("Kyle", 4); ("Stan", 3); ("Keny", 3)]


Situations similaires du paradoxe d'Arrow:

Avant que Keny se présente à l'éléction, Stan était passé au second tour avec 5 voix, devant Kyle. Sauf qu'à cause de Keny, les votes ont été départagés et donc Kyle est passé au second tour. On peut faire le lien avec les éléctions présidentielles de 2017. En effet, sans la candidature de Benoit Hamon (Parti Socialiste), Jean-Luc Mélenchon (France Insoumise) pouvait arriver au second tour.
On peut également faire le lien avec les éléctions présidentielles de 2022, car il y a eu de nombreux candidats de partis politiques de gauche
qui se sont présentés, induisant la dispersion du vote, et donc la non représentation du parti de gauche au second tour, car cette dispersion a fait passé le parti de Macron (Centre) et le parti de Le Pen (droite).

### Question 9

La source du problème dans le scrutin uninominal est dans la division au sein des partis par la candidature de nombreux candidats qui divisent donc les partis politiques en plusieurs camps. 

## 3. Jugement majoritaire

### Question 10

Dans le cas d'une élection avec 12 candidats, chaque électeur peut donner une mention à chaque candidat parmi les 6 mentions possibles. Par conséquent, chaque électeur a 6 choix possibles pour chaque candidat, soit un total de 6^12 = 2 176 782 336 bulletins différents possibles.

En comparaison, dans un scrutin uninominal (où chaque électeur ne vote que pour un seul candidat), il n'y a que 13 possibilités (12 candidats plus la possibilité de voter blanc ou nul).

Le jugement majoritaire offre donc beaucoup plus de choix aux électeurs que le scrutin uninominal. Cependant, cela peut rendre le dépouillement plus complexe et nécessiter plus de ressources pour effectuer le comptage des voix.

### Question 11

In [20]:
(* Defining types *)
type mention = Arejeter | Insuffisant | Passable | Assezbien | Bien | Tresbien ;;
type bulletin_jm = mention list ;;
type urne_jm = bulletin_jm list ;;
type nombre = Un | Deux | Trois | Quatre | Cinq | Six | Sept | Huit | Neuf | Dix | Onze | Douze ;;

type mention =
    Arejeter
  | Insuffisant
  | Passable
  | Assezbien
  | Bien
  | Tresbien


type bulletin_jm = mention list


type urne_jm = bulletin_jm list


type nombre =
    Un
  | Deux
  | Trois
  | Quatre
  | Cinq
  | Six
  | Sept
  | Huit
  | Neuf
  | Dix
  | Onze
  | Douze


In [21]:
let u = [[Tresbien; Assezbien; Arejeter; Passable]; (* Premier bulletin *)
[Assezbien; Assezbien; Arejeter; Tresbien]; (* Second bulletin *)
[Tresbien; Arejeter; Arejeter; Tresbien]] (* Troisieme bulletin *)


val u : mention list list =
  [[Tresbien; Assezbien; Arejeter; Passable];
   [Assezbien; Assezbien; Arejeter; Tresbien];
   [Tresbien; Arejeter; Arejeter; Tresbien]]


### Question 12

In [22]:
let rec depouiller_jm (u:urne_jm) : mention list list = 
    match u with
        |[] -> []
        |[]::lp -> []
        |lp -> let lh = List.fold_right (fun x acc -> (List.hd x)::acc) lp [] in
                    let lt = List.fold_right (fun x acc -> (List.tl x)::acc) lp [] in 
                        lh::(depouiller_jm lt) ;;
                        
let ms = depouiller_jm u  ;;

val depouiller_jm : urne_jm -> mention list list = <fun>


val ms : mention list list =
  [[Tresbien; Assezbien; Tresbien]; [Assezbien; Assezbien; Arejeter];
   [Arejeter; Arejeter; Arejeter]; [Passable; Tresbien; Tresbien]]


### Question 13

In [23]:
let tri (l: 'a list) : 'a list =
    List.sort compare l ;;

let tri_mentions (u: urne_jm) : bulletin_jm list =
  List.map tri u ;;

let ms_triee = tri_mentions ms ;;


val tri : 'a list -> 'a list = <fun>


val tri_mentions : urne_jm -> bulletin_jm list = <fun>


val ms_triee : bulletin_jm list =
  [[Assezbien; Tresbien; Tresbien]; [Arejeter; Assezbien; Assezbien];
   [Arejeter; Arejeter; Arejeter]; [Passable; Tresbien; Tresbien]]


### Question 14

In [24]:
let mediane (b : bulletin_jm) : mention =
    let len = List.length b in
        List.nth b (len/2) ;;                

mediane [Arejeter; Arejeter; Assezbien; Bien; Tresbien] ;;

val mediane : bulletin_jm -> mention = <fun>


- : mention = Assezbien


### Question 15

In [25]:
(*Algorithme de la fonction: 
    - crée une liste des medianes 
    - tri cette liste allant de pire à meilleure mediane
    - renvoie le premier élément de la liste inversée (c'est-à-dire la meilleure médiane)*)
let meilleure_mediane (ms: bulletin_jm list) : mention =
      let medianes = List.map mediane ms in 
          let s = List.sort compare medianes in 
          List.hd (List.rev s);;
          
(*Test avec jeux d'essais*)
meilleure_mediane ms_triee;;

val meilleure_mediane : bulletin_jm list -> mention = <fun>


- : mention = Tresbien


### Question 16

In [26]:
let supprime_perdants (ms: bulletin_jm list) : bulletin_jm list =
  let meilleure = meilleure_mediane ms in
      List.map (fun b -> if mediane b = meilleure then b else []) ms ;;

let ms_sup = supprime_perdants ms_triee ;;

val supprime_perdants : bulletin_jm list -> bulletin_jm list = <fun>


val ms_sup : bulletin_jm list =
  [[Assezbien; Tresbien; Tresbien]; []; []; [Passable; Tresbien; Tresbien]]


### Question 17

In [27]:
let rec supprime_mention (l:bulletin_jm) (m: mention) : bulletin_jm = 
    match l with 
        [] -> []
        | e::remainder -> if e = m then remainder else e::(supprime_mention remainder m) ;;
        
(*Test avec jeu d'essai*)        
supprime_mention [Assezbien ; Tresbien ; Tresbien] Tresbien ;;

val supprime_mention : bulletin_jm -> mention -> bulletin_jm = <fun>


- : bulletin_jm = [Assezbien; Tresbien]


In [28]:
let supprime_meilleure_mediane (l:urne_jm):urne_jm =
  let non_empty_lists = List.filter (fun x -> x <> []) l in
  let best_median = meilleure_mediane non_empty_lists in
  List.map (fun lst -> if lst = [] then [] else supprime_mention lst best_median) l ;;
  
let ms_triee2 = supprime_meilleure_mediane ms_sup ;;

val supprime_meilleure_mediane : urne_jm -> urne_jm = <fun>


val ms_triee2 : urne_jm =
  [[Assezbien; Tresbien]; []; []; [Passable; Tresbien]]


### Question 18

In [29]:
let vainqueur_jm (ms : bulletin_jm list) : candidat =
  let candidats = List.init (List.length ms) (fun i -> i) in
      let candidats_filtres = List.filter (fun i -> (List.nth ms i) != []) candidats in
          match candidats_filtres with
              | [i] -> string_of_int i
              | _ ->
                let medianes = List.map (fun i -> mediane (List.nth ms i)) candidats_filtres in
                    let mediane_max = List.fold_left max Arejeter medianes in
                        let max_count = List.fold_left (fun acc m -> if m = mediane_max then acc + 1 else acc) 0 medianes in
                            if max_count = 1 then string_of_int (List.find (fun i -> mediane (List.nth ms i) = mediane_max) candidats_filtres) else ""
;;

vainqueur_jm ms_triee2 ;;
vainqueur_jm [[];[Passable]; []; []];;
vainqueur_jm [[Passable; Tresbien; Tresbien]; [Arejeter; Bien; Tresbien];
   [Arejeter; Arejeter; Passable]];;

val vainqueur_jm : bulletin_jm list -> candidat = <fun>


- : candidat = ""


- : candidat = "1"


- : candidat = "0"


### Question 19

In [30]:
(*3 candidats*)
let lc1:panel = ["Eric";"Kyle";"Stan"];;
let u1:urne = ["Eric";"Kyle";"Stan";"Kyle";"Kyle";"Stan";"Eric";"Eric";"Kyle";"Eric";"Stan";"Eric";"Eric";"Eric";"Stan";"Stan"];;

(*4 candidats*)
let lc2:panel = ["Eric";"Kyle";"Stan";"Keny"];;
let u2:urne = ["Keny";"Kyle";"Keny";"Kyle";"Kyle";"Keny";"Eric";"Eric";"Kyle";"Eric";"Stan";"Eric";"Eric";"Eric";"Stan";"Stan"];;

(* Urne jugement majoritaire *)
let (b1:bulletin_jm) = [Tresbien;Assezbien;Arejeter;Passable];;
let (b2:bulletin_jm) = [Assezbien;Assezbien;Arejeter;Tresbien];;
let (b3:bulletin_jm) = [Tresbien;Arejeter;Arejeter;Tresbien];;

type urne_jm = bulletin_jm list;;

let (ujm1:urne_jm) = [b1;b2;b3];;
let (ujm2:urne_jm) =
  [[Bien; Assezbien; Insuffisant; Bien]; [Bien; Bien; Passable; Bien];
   [Assezbien; Assezbien; Insuffisant; Tresbien];
   [Arejeter; Assezbien; Insuffisant; Bien];
   [Assezbien; Arejeter; Assezbien; Bien];
   [Arejeter; Assezbien; Tresbien; Tresbien];
   [Assezbien; Arejeter; Insuffisant; Bien];
   [Passable; Passable; Insuffisant; Bien];
   [Insuffisant; Bien; Tresbien; Bien];
   [Arejeter; Assezbien; Passable; Tresbien];
   [Bien; Assezbien; Assezbien; Bien]; [Tresbien; Tresbien; Tresbien; Bien];
   [Passable; Tresbien; Insuffisant; Tresbien];
   [Insuffisant; Bien; Assezbien; Bien];
   [Arejeter; Passable; Insuffisant; Tresbien];
   [Arejeter; Tresbien; Assezbien; Tresbien];
   [Insuffisant; Passable; Bien; Tresbien];
   [Assezbien; Assezbien; Passable; Tresbien];
   [Passable; Insuffisant; Bien; Bien];
   [Assezbien; Insuffisant; Passable; Tresbien];
   [Tresbien; Arejeter; Insuffisant; Tresbien];
   [Passable; Arejeter; Bien; Tresbien];
   [Assezbien; Bien; Passable; Tresbien];
   [Tresbien; Tresbien; Assezbien; Tresbien];
   [Tresbien; Arejeter; Assezbien; Tresbien]; [Bien; Bien; Passable; Bien];
   [Tresbien; Arejeter; Passable; Bien];
   [Assezbien; Bien; Passable; Tresbien]; [Tresbien; Bien; Tresbien; Bien];
   [Arejeter; Arejeter; Tresbien; Bien];
   [Assezbien; Tresbien; Assezbien; Tresbien];
   [Passable; Tresbien; Bien; Tresbien];
   [Insuffisant; Bien; Arejeter; Tresbien];
   [Tresbien; Tresbien; Arejeter; Bien]; [Arejeter; Bien; Bien; Tresbien];
   [Arejeter; Assezbien; Assezbien; Tresbien];
   [Arejeter; Bien; Tresbien; Tresbien];
   [Passable; Bien; Arejeter; Tresbien];
   [Insuffisant; Tresbien; Insuffisant; Tresbien];
   [Assezbien; Bien; Insuffisant; Tresbien];
   [Passable; Insuffisant; Insuffisant; Tresbien];
   [Assezbien; Insuffisant; Passable; Tresbien];
   [Assezbien; Assezbien; Bien; Bien]; [Assezbien; Passable; Tresbien; Bien];
   [Insuffisant; Arejeter; Bien; Tresbien];
   [Assezbien; Bien; Passable; Tresbien];
   [Bien; Insuffisant; Arejeter; Tresbien]; [Passable; Bien; Tresbien; Bien];
   [Assezbien; Assezbien; Tresbien; Tresbien];
   [Tresbien; Bien; Tresbien; Tresbien];
   [Tresbien; Assezbien; Tresbien; Bien];
   [Arejeter; Assezbien; Insuffisant; Bien];
   [Assezbien; Insuffisant; Tresbien; Tresbien];
   [Arejeter; Bien; Passable; Tresbien]; [Assezbien; Arejeter; Bien; Bien];
   [Arejeter; Assezbien; Insuffisant; Bien];
   [Arejeter; Passable; Passable; Tresbien];
   [Assezbien; Bien; Tresbien; Tresbien];
   [Tresbien; Arejeter; Bien; Tresbien];
   [Assezbien; Tresbien; Bien; Tresbien];
   [Passable; Arejeter; Arejeter; Tresbien];
   [Arejeter; Passable; Insuffisant; Bien];
   [Tresbien; Passable; Assezbien; Tresbien];
   [Assezbien; Insuffisant; Arejeter; Bien];
   [Insuffisant; Tresbien; Assezbien; Tresbien];
   [Insuffisant; Bien; Insuffisant; Bien]; [Tresbien; Tresbien; Bien; Bien];
   [Insuffisant; Insuffisant; Bien; Bien];
   [Passable; Arejeter; Assezbien; Bien];
   [Tresbien; Tresbien; Insuffisant; Bien];
   [Tresbien; Passable; Tresbien; Bien];
   [Assezbien; Tresbien; Passable; Bien]; [Bien; Assezbien; Assezbien; Bien];
   [Insuffisant; Arejeter; Passable; Tresbien];
   [Tresbien; Bien; Arejeter; Bien];
   [Arejeter; Arejeter; Passable; Tresbien];
   [Arejeter; Bien; Assezbien; Bien]; [Arejeter; Passable; Assezbien; Bien];
   [Tresbien; Tresbien; Passable; Bien]; [Bien; Insuffisant; Passable; Bien];
   [Bien; Tresbien; Bien; Tresbien]; [Tresbien; Bien; Passable; Bien];
   [Passable; Assezbien; Insuffisant; Tresbien];
   [Bien; Arejeter; Insuffisant; Tresbien];
   [Assezbien; Passable; Assezbien; Bien];
   [Assezbien; Arejeter; Tresbien; Bien];
   [Tresbien; Insuffisant; Arejeter; Tresbien];
   [Arejeter; Insuffisant; Arejeter; Bien];
   [Tresbien; Bien; Passable; Tresbien];
   [Assezbien; Arejeter; Arejeter; Tresbien];
   [Insuffisant; Arejeter; Insuffisant; Bien];
   [Assezbien; Insuffisant; Arejeter; Bien];
   [Assezbien; Tresbien; Arejeter; Tresbien];
   [Passable; Arejeter; Tresbien; Bien];
   [Insuffisant; Arejeter; Insuffisant; Bien];
   [Insuffisant; Passable; Tresbien; Tresbien];
   [Passable; Arejeter; Arejeter; Tresbien];
   [Arejeter; Arejeter; Bien; Tresbien]; [Arejeter; Insuffisant; Bien; Bien];
   [Arejeter; Insuffisant; Arejeter; Bien]];;

let (ujm3:urne_jm) = 
  [[Arejeter; Bien; Tresbien; Insuffisant; Bien; Assezbien];
   [Insuffisant; Bien; Bien; Insuffisant; Passable; Assezbien];
   [Assezbien; Assezbien; Tresbien; Arejeter; Assezbien; Tresbien];
   [Bien; Assezbien; Bien; Passable; Insuffisant; Passable];
   [Insuffisant; Arejeter; Bien; Bien; Arejeter; Tresbien];
   [Assezbien; Arejeter; Tresbien; Tresbien; Arejeter; Bien];
   [Arejeter; Tresbien; Bien; Passable; Passable; Insuffisant];
   [Passable; Passable; Bien; Bien; Bien; Passable];
   [Insuffisant; Tresbien; Bien; Arejeter; Tresbien; Tresbien];
   [Arejeter; Passable; Tresbien; Assezbien; Bien; Assezbien];
   [Assezbien; Passable; Bien; Assezbien; Insuffisant; Arejeter];
   [Arejeter; Insuffisant; Bien; Passable; Insuffisant; Arejeter];
   [Insuffisant; Arejeter; Bien; Bien; Arejeter; Assezbien];
   [Assezbien; Bien; Tresbien; Bien; Arejeter; Tresbien];
   [Passable; Passable; Tresbien; Assezbien; Assezbien; Bien];
   [Arejeter; Assezbien; Bien; Tresbien; Assezbien; Bien];
   [Passable; Arejeter; Tresbien; Tresbien; Assezbien; Bien];
   [Assezbien; Tresbien; Tresbien; Assezbien; Insuffisant; Assezbien];
   [Tresbien; Assezbien; Tresbien; Arejeter; Insuffisant; Bien];
   [Bien; Tresbien; Bien; Arejeter; Bien; Insuffisant];
   [Insuffisant; Assezbien; Tresbien; Insuffisant; Bien; Bien];
   [Arejeter; Insuffisant; Tresbien; Arejeter; Assezbien; Tresbien];
   [Insuffisant; Tresbien; Tresbien; Arejeter; Arejeter; Insuffisant];
   [Passable; Bien; Tresbien; Bien; Arejeter; Assezbien];
   [Insuffisant; Assezbien; Tresbien; Passable; Passable; Arejeter];
   [Assezbien; Arejeter; Bien; Bien; Assezbien; Passable];
   [Arejeter; Bien; Bien; Bien; Bien; Tresbien];
   [Tresbien; Assezbien; Bien; Tresbien; Passable; Bien];
   [Bien; Tresbien; Tresbien; Arejeter; Bien; Insuffisant];
   [Arejeter; Arejeter; Tresbien; Passable; Tresbien; Bien];
   [Arejeter; Assezbien; Bien; Insuffisant; Insuffisant; Insuffisant];
   [Assezbien; Passable; Bien; Tresbien; Bien; Assezbien];
   [Passable; Bien; Bien; Bien; Passable; Assezbien];
   [Tresbien; Arejeter; Tresbien; Assezbien; Assezbien; Bien];
   [Insuffisant; Passable; Bien; Bien; Tresbien; Bien];
   [Tresbien; Insuffisant; Bien; Bien; Arejeter; Bien];
   [Arejeter; Tresbien; Tresbien; Passable; Assezbien; Bien];
   [Arejeter; Assezbien; Bien; Assezbien; Assezbien; Arejeter];
   [Bien; Tresbien; Tresbien; Passable; Insuffisant; Bien];
   [Tresbien; Arejeter; Tresbien; Assezbien; Passable; Passable];
   [Passable; Tresbien; Tresbien; Passable; Assezbien; Assezbien];
   [Bien; Bien; Tresbien; Insuffisant; Assezbien; Passable];
   [Passable; Passable; Tresbien; Insuffisant; Passable; Tresbien];
   [Assezbien; Passable; Bien; Insuffisant; Passable; Bien];
   [Passable; Tresbien; Tresbien; Assezbien; Arejeter; Assezbien];
   [Bien; Passable; Bien; Assezbien; Arejeter; Insuffisant];
   [Assezbien; Insuffisant; Bien; Passable; Tresbien; Passable];
   [Bien; Arejeter; Bien; Insuffisant; Assezbien; Assezbien];
   [Bien; Bien; Bien; Assezbien; Tresbien; Insuffisant];
   [Assezbien; Arejeter; Bien; Passable; Assezbien; Bien];
   [Tresbien; Passable; Bien; Assezbien; Bien; Tresbien];
   [Bien; Tresbien; Tresbien; Assezbien; Bien; Passable];
   [Passable; Insuffisant; Tresbien; Bien; Insuffisant; Tresbien];
   [Bien; Assezbien; Tresbien; Tresbien; Bien; Arejeter];
   [Bien; Bien; Bien; Passable; Insuffisant; Passable];
   [Bien; Tresbien; Tresbien; Assezbien; Passable; Bien];
   [Insuffisant; Passable; Bien; Passable; Tresbien; Arejeter];
   [Bien; Insuffisant; Bien; Passable; Tresbien; Arejeter];
   [Passable; Assezbien; Tresbien; Tresbien; Assezbien; Passable];
   [Passable; Assezbien; Tresbien; Passable; Arejeter; Tresbien];
   [Insuffisant; Tresbien; Tresbien; Assezbien; Assezbien; Assezbien];
   [Bien; Tresbien; Tresbien; Bien; Assezbien; Insuffisant];
   [Arejeter; Assezbien; Bien; Assezbien; Tresbien; Arejeter];
   [Bien; Tresbien; Tresbien; Tresbien; Arejeter; Arejeter];
   [Insuffisant; Tresbien; Bien; Arejeter; Insuffisant; Insuffisant];
   [Bien; Arejeter; Tresbien; Insuffisant; Passable; Passable];
   [Insuffisant; Insuffisant; Tresbien; Passable; Arejeter; Bien];
   [Insuffisant; Insuffisant; Tresbien; Tresbien; Tresbien; Insuffisant];
   [Tresbien; Passable; Tresbien; Passable; Passable; Passable];
   [Passable; Insuffisant; Bien; Tresbien; Tresbien; Tresbien];
   [Bien; Passable; Tresbien; Passable; Passable; Tresbien];
   [Passable; Arejeter; Tresbien; Assezbien; Assezbien; Passable];
   [Bien; Assezbien; Tresbien; Insuffisant; Arejeter; Tresbien];
   [Bien; Tresbien; Tresbien; Insuffisant; Passable; Assezbien];
   [Arejeter; Assezbien; Bien; Arejeter; Bien; Assezbien];
   [Bien; Tresbien; Bien; Tresbien; Tresbien; Tresbien];
   [Passable; Arejeter; Bien; Passable; Assezbien; Tresbien];
   [Assezbien; Passable; Bien; Tresbien; Assezbien; Assezbien];
   [Insuffisant; Tresbien; Bien; Arejeter; Tresbien; Bien];
   [Insuffisant; Bien; Bien; Bien; Bien; Arejeter];
   [Passable; Bien; Bien; Insuffisant; Tresbien; Insuffisant];
   [Passable; Passable; Tresbien; Assezbien; Tresbien; Arejeter];
   [Tresbien; Assezbien; Bien; Assezbien; Insuffisant; Bien];
   [Arejeter; Passable; Tresbien; Insuffisant; Tresbien; Passable];
   [Tresbien; Passable; Bien; Arejeter; Insuffisant; Insuffisant];
   [Tresbien; Bien; Tresbien; Passable; Insuffisant; Insuffisant];
   [Tresbien; Arejeter; Tresbien; Assezbien; Passable; Assezbien];
   [Assezbien; Bien; Tresbien; Passable; Tresbien; Arejeter];
   [Bien; Arejeter; Bien; Insuffisant; Tresbien; Tresbien];
   [Insuffisant; Arejeter; Tresbien; Bien; Tresbien; Passable];
   [Bien; Insuffisant; Tresbien; Passable; Tresbien; Assezbien];
   [Assezbien; Passable; Bien; Insuffisant; Bien; Passable];
   [Insuffisant; Assezbien; Tresbien; Bien; Bien; Assezbien];
   [Bien; Tresbien; Bien; Bien; Assezbien; Tresbien];
   [Insuffisant; Bien; Tresbien; Tresbien; Passable; Arejeter];
   [Passable; Insuffisant; Tresbien; Bien; Insuffisant; Arejeter];
   [Passable; Arejeter; Bien; Arejeter; Passable; Assezbien];
   [Passable; Arejeter; Tresbien; Assezbien; Tresbien; Arejeter];
   [Arejeter; Assezbien; Bien; Passable; Tresbien; Bien];
   [Bien; Bien; Bien; Assezbien; Tresbien; Passable]];;


val lc1 : panel = ["Eric"; "Kyle"; "Stan"]


val u1 : urne =
  ["Eric"; "Kyle"; "Stan"; "Kyle"; "Kyle"; "Stan"; "Eric"; "Eric"; "Kyle";
   "Eric"; "Stan"; "Eric"; "Eric"; "Eric"; "Stan"; "Stan"]


val lc2 : panel = ["Eric"; "Kyle"; "Stan"; "Keny"]


val u2 : urne =
  ["Keny"; "Kyle"; "Keny"; "Kyle"; "Kyle"; "Keny"; "Eric"; "Eric"; "Kyle";
   "Eric"; "Stan"; "Eric"; "Eric"; "Eric"; "Stan"; "Stan"]


val b1 : bulletin_jm = [Tresbien; Assezbien; Arejeter; Passable]


...

val b3 : bulletin_jm = [Tresbien; Arejeter; Arejeter; Tresbien]


type urne_jm = bulletin_jm list


val ujm1 : urne_jm =
  [[Tresbien; Assezbien; Arejeter; Passable];
   [Assezbien; Assezbien; Arejeter; Tresbien];
   [Tresbien; Arejeter; Arejeter; Tresbien]]


val ujm2 : urne_jm =
  [[Bien; Assezbien; Insuffisant; Bien]; [Bien; Bien; Passable; Bien];
   [Assezbien; Assezbien; Insuffisant; Tresbien];
   [Arejeter; Assezbien; Insuffisant; Bien];
   [Assezbien; Arejeter; Assezbien; Bien];
   [Arejeter; Assezbien; Tresbien; Tresbien];
   [Assezbien; Arejeter; Insuffisant; Bien];
   [Passable; Passable; Insuffisant; Bien];
   [Insuffisant; Bien; Tresbien; Bien];
   [Arejeter; Assezbien; Passable; Tresbien];
   [Bien; Assezbien; Assezbien; Bien]; [Tresbien; Tresbien; Tresbien; Bien];
   [Passable; Tresbien; Insuffisant; Tresbien];
   [Insuffisant; Bien; Assezbien; Bien];
   [Arejeter; Passable; Insuffisant; Tresbien];
   [Arejeter; Tresbien; Assezbien; Tresbien];
   [Insuffisant; Passable; Bien; Tresbien];
   [Assezbien; Assezbien; Passable; Tresbien];
   [Passable; Insuffisant; Bien; Bien];
   [Assezbien; Insuffisant; Passable; Tresbien];
   [Tresbien; Arejeter; Insuffisant; Tresbien];
   [Passable; Arejeter; Bien; Tresbien];
   [Assezbien; Bien;

val ujm3 : urne_jm =
  [[Arejeter; Bien; Tresbien; Insuffisant; Bien; Assezbien];
   [Insuffisant; Bien; Bien; Insuffisant; Passable; Assezbien];
   [Assezbien; Assezbien; Tresbien; Arejeter; Assezbien; Tresbien];
   [Bien; Assezbien; Bien; Passable; Insuffisant; Passable];
   [Insuffisant; Arejeter; Bien; Bien; Arejeter; Tresbien];
   [Assezbien; Arejeter; Tresbien; Tresbien; Arejeter; Bien];
   [Arejeter; Tresbien; Bien; Passable; Passable; Insuffisant];
   [Passable; Passable; Bien; Bien; Bien; Passable];
   [Insuffisant; Tresbien; Bien; Arejeter; Tresbien; Tresbien];
   [Arejeter; Passable; Tresbien; Assezbien; Bien; Assezbien];
   [Assezbien; Passable; Bien; Assezbien; Insuffisant; Arejeter];
   [Arejeter; Insuffisant; Bien; Passable; Insuffisant; Arejeter];
   [Insuffisant; Arejeter; Bien; Bien; Arejeter; Assezbien];
   [Assezbien; Bien; Tresbien; Bien; Arejeter; Tresbien];
   [Passable; Passable; Tresbien; Assezbien; Assezbien; Bien];
   [Arejeter; Assezbien; Bien; Tresbien; Ass

In [31]:
let rec trouve_vainqueur_jm (urne: urne_jm) : string =
    let resultat = vainqueur_jm urne in
        if resultat <> "" then resultat else
            let urne_meilleure_mediane_supprimee = supprime_meilleure_mediane urne in
                trouve_vainqueur_jm urne_meilleure_mediane_supprimee ;; 

trouve_vainqueur_jm ujm1 ;;
trouve_vainqueur_jm ujm2 ;;
trouve_vainqueur_jm ujm3 ;;

val trouve_vainqueur_jm : urne_jm -> candidat = <fun>


- : candidat = "1"


- : candidat = "56"


- : candidat = "75"


### Question 20 

D'un côté, le jugement majoritaire résout le problème du vote utile car il permet de donner aux électeurs une évaluation plus nuancée des candidats plutôt que de devoir choisir un seul candidat. Cela donne donc l'opportunité aux électeurs de donner leur avis sur les candidats qui peut être à la fois positif ou négatif. Cela incite donc plus à voter car donne non seulement l'impression que l'avis de l'électeur est profondément pris en compte, mais aussi qu'il a un réel impact sur la décision de l'élection. 

Cependant, le jugement majoritaire peut entraîner des résultats incohérents voir louphoques. En effet, la manière de "noter" de chacun peut différer car peut être basée sur des critères très différents, c'est donc des votes assez subjectifs. 

## 4. Recomptons les voix

### Question 21

In [32]:
type bv_list = (string * int) list;; (*type bureau de vote list qui modélise une liste tel que [("ROUSSEL", 595)]*)
type ville = string ;;
type zone = Reg of string | Dpt of string ;;
    
type arbre =
  | Empty
  | N of zone * arbre list
  | Bv of ville * bv_list;;



type bv_list = (string * int) list


type ville = string


type zone = Reg of string | Dpt of string


type arbre = Empty | N of zone * arbre list | Bv of ville * bv_list


In [33]:
let ara =
N (Reg "Auvergne-Rhône-Alpes",
 [N (Dpt "Drôme",
   [Bv ("Valence",
     [("ARTHAUD", 161); ("ROUSSEL", 595); ("MACRON", 7756); ("LASSALLE", 590);
      ("LE PEN", 4679); ("ZEMMOUR", 2080); ("MÉLENCHON", 8398);
      ("HIDALGO", 519); ("JADOT", 1701); ("PÉCRESSE", 1423); ("POUTOU", 186);
      ("DUPONT-AIGNAN", 573)]);
    Bv ("Romans-sur-Isère",
     [("ARTHAUD", 181); ("ROUSSEL", 371); ("MACRON", 4030); ("LASSALLE", 334);
      ("LE PEN", 3270); ("ZEMMOUR", 1072); ("MÉLENCHON", 4108);
      ("HIDALGO", 251); ("JADOT", 850); ("PÉCRESSE", 631); ("POUTOU", 111);
      ("DUPONT-AIGNAN", 341)])]);
  N (Dpt "Isère",
   [Bv ("Meylan",
     [("ARTHAUD", 28); ("ROUSSEL", 169); ("MACRON", 4457); ("LASSALLE", 164);
      ("LE PEN", 1288); ("ZEMMOUR", 928); ("MÉLENCHON", 2198);
      ("HIDALGO", 251); ("JADOT", 906); ("PÉCRESSE", 763); ("POUTOU", 64);
      ("DUPONT-AIGNAN", 162)]);
    Bv ("Echirolles",
     [("ARTHAUD", 104); ("ROUSSEL", 506); ("MACRON", 3276); ("LASSALLE", 259);
      ("LE PEN", 2737); ("ZEMMOUR", 779); ("MÉLENCHON", 5121);
      ("HIDALGO", 223); ("JADOT", 590); ("PÉCRESSE", 360); ("POUTOU", 92);
      ("DUPONT-AIGNAN", 202)]);
    Bv ("Fontaine",
     [("ARTHAUD", 55); ("ROUSSEL", 363); ("MACRON", 2111); ("LASSALLE", 146);
      ("LE PEN", 1835); ("ZEMMOUR", 541); ("MÉLENCHON", 3113);
      ("HIDALGO", 185); ("JADOT", 493); ("PÉCRESSE", 212); ("POUTOU", 83);
      ("DUPONT-AIGNAN", 121)]);
    Bv ("Saint-Martin-d'Hères",
     [("ARTHAUD", 58); ("ROUSSEL", 436); ("MACRON", 2769); ("LASSALLE", 207);
      ("LE PEN", 2289); ("ZEMMOUR", 661); ("MÉLENCHON", 4763);
      ("HIDALGO", 242); ("JADOT", 777); ("PÉCRESSE", 300); ("POUTOU", 119);
      ("DUPONT-AIGNAN", 161)]);
    Bv ("Gières",
     [("ARTHAUD", 16); ("ROUSSEL", 66); ("MACRON", 1071); ("LASSALLE", 84);
      ("LE PEN", 641); ("ZEMMOUR", 205); ("MÉLENCHON", 844); ("HIDALGO", 96);
      ("JADOT", 301); ("PÉCRESSE", 155); ("POUTOU", 30);
      ("DUPONT-AIGNAN", 61)]);
    Bv ("Grenoble",
     [("ARTHAUD", 256); ("ROUSSEL", 1300); ("MACRON", 15968);
      ("LASSALLE", 845); ("LE PEN", 6444); ("ZEMMOUR", 3389);
      ("MÉLENCHON", 24568); ("HIDALGO", 1488); ("JADOT", 5644);
      ("PÉCRESSE", 2019); ("POUTOU", 508); ("DUPONT-AIGNAN", 661)])])]);;
    
let panel_2022 = ["ARTHAUD";"ROUSSEL";"MACRON";"LASSALLE";"LE PEN";"ZEMMOUR";"MÉLENCHON";"HIDALGO";"JADOT";"PÉCRESSE";"POUTOU";"DUPONT-AIGNAN"];;


val ara : arbre =
  N (Reg "Auvergne-Rhône-Alpes",
   [N (Dpt "Drôme",
     [Bv ("Valence",
       [("ARTHAUD", 161); ("ROUSSEL", 595); ("MACRON", 7756);
        ("LASSALLE", 590); ("LE PEN", 4679); ("ZEMMOUR", 2080);
        ("MÉLENCHON", 8398); ("HIDALGO", 519); ("JADOT", 1701);
        ("PÉCRESSE", 1423); ("POUTOU", 186); ("DUPONT-AIGNAN", 573)]);
      Bv ("Romans-sur-Isère",
       [("ARTHAUD", 181); ("ROUSSEL", 371); ("MACRON", 4030);
        ("LASSALLE", 334); ("LE PEN", 3270); ("ZEMMOUR", 1072);
        ("MÉLENCHON", 4108); ("HIDALGO", 251); ("JADOT", 850);
        ("PÉCRESSE", 631); ("POUTOU", 111); ("DUPONT-AIGNAN", 341)])]);
    N (Dpt "Isère",
     [Bv ("Meylan",
       [("ARTHAUD", 28); ("ROUSSEL", 169); ("MACRON", 4457);
        ("LASSALLE", 164); ("LE PEN", 1288); ("ZEMMOUR", 928);
        ("MÉLENCHON", 2198); ("HIDALGO", 251); ("JADOT", 906);
        ("PÉCRESSE", 763); ("POUTOU", 64); ("DUPONT-AIGNAN", 162)]);
      Bv ("Echirolles",
       [("ARTHAUD", 104); ("ROUSSEL"

val panel_2022 : string list =
  ["ARTHAUD"; "ROUSSEL"; "MACRON"; "LASSALLE"; "LE PEN"; "ZEMMOUR";
   "MÉLENCHON"; "HIDALGO"; "JADOT"; "PÉCRESSE"; "POUTOU"; "DUPONT-AIGNAN"]


### Question 22

In [34]:
let rec trouve_bv (arbre: arbre) (ville: ville) : bv_list =
  match arbre with
  | Empty -> []
  | Bv (nom, candidats) -> if nom = ville then candidats else []
  | N (_, sous_arbres) ->
      let rec trouver_ville arbres =
        match arbres with
        | [] -> []
        | arb :: rem ->
            let candidats_v = trouve_bv arb ville in
            if candidats_v <> [] then candidats_v else trouver_ville rem
      in
      trouver_ville sous_arbres ;;


val trouve_bv : arbre -> ville -> bv_list = <fun>


In [35]:
trouve_bv ara "Valence" ;;
trouve_bv ara "Grenoble" ;;
trouve_bv ara "Fontaine" ;;

- : bv_list =
[("ARTHAUD", 161); ("ROUSSEL", 595); ("MACRON", 7756); ("LASSALLE", 590);
 ("LE PEN", 4679); ("ZEMMOUR", 2080); ("MÉLENCHON", 8398); ("HIDALGO", 519);
 ("JADOT", 1701); ("PÉCRESSE", 1423); ("POUTOU", 186);
 ("DUPONT-AIGNAN", 573)]


- : bv_list =
[("ARTHAUD", 256); ("ROUSSEL", 1300); ("MACRON", 15968); ("LASSALLE", 845);
 ("LE PEN", 6444); ("ZEMMOUR", 3389); ("MÉLENCHON", 24568);
 ("HIDALGO", 1488); ("JADOT", 5644); ("PÉCRESSE", 2019); ("POUTOU", 508);
 ("DUPONT-AIGNAN", 661)]


- : bv_list =
[("ARTHAUD", 55); ("ROUSSEL", 363); ("MACRON", 2111); ("LASSALLE", 146);
 ("LE PEN", 1835); ("ZEMMOUR", 541); ("MÉLENCHON", 3113); ("HIDALGO", 185);
 ("JADOT", 493); ("PÉCRESSE", 212); ("POUTOU", 83); ("DUPONT-AIGNAN", 121)]


### Question 23

In [36]:
let resultat = union (trouve_bv ara "Valence") (trouve_bv ara "Fontaine") in
    let res = union resultat (trouve_bv ara "Grenoble") in
        max_depouille res ;;

- : candidat * score = ("MÉLENCHON", 36079)


## 5. Conclusion

En conclusion, les méthodes de scrutin uninominal et du jugement majoritaire admettent tous deux des défauts.

D'un côté, le scrutin uninominal créée une division au sein des partis, induisant un résultat d'élection qui ne satisfait pas forcément la majorité de la population, en parti à cause du "vote utile".

D'un autre côté, certe le jugement majoritaire résout le problème du "vote utile", mais il accompagne avec lui une certaine subjectivité au vote qui peut avoir des effets néfastes envers une élection en créant des résulats incohérents.